# Exercise 1.4 - Solution
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
# Pacotes usados
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from scipy.stats.stats import spearmanr
from scipy.stats.stats import pearsonr
from IPython.display import display, Markdown

# Código da aula mantido para nossa solução:

In [ ]:
# Data source:
# https://archive.ics.uci.edu/ml/datasets/Adult

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
# Arquivo texto, delimitado por vírgulas, sem cabeçalho

Descrição do arquivo em:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

### Atributos:

1. **age**: continuous.
* **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* **fnlwgt**: continuous.
* **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* **education-num**: continuous.
* **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* **sex**: Female, Male.
* **capital-gain**: continuous.
* **capital-loss**: continuous.
* **hours-per-week**: continuous.
* **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
* **income**: >50K, <=50K.


In [ ]:
# lê arquivo texto da internet, linha a linha
# Nino OBS.: Tive que incluir request no import abaixo pra poder fazer a chamada urllib.request no for
import urllib.request

data_raw=[]

for line in urllib.request.urlopen(url):
    # from bytes-like object to string
    lined = line.decode()
    # separa cada linha em campos (separados por vírgula)
    field_lst = str(lined).split(', ')
    
    # inclui no data frame se o número de atributos for o esperado
    if len(field_lst)==15:
        data_raw.append(field_lst[0:15])
    else:
        print('skipped data (error? blank line?):', line)


In [ ]:
# cria dataframe pandas com dados


df = pd.DataFrame(data_raw)
df.shape

In [ ]:
# define colunas conforme definição do data set

df.columns = ['age', 'type_employer', 'fnlwgt', 'education', 
                "education_num","marital", "occupation", "relationship", "race","sex",
                "capital_gain", "capital_loss", "hr_per_week","country","income"]

In [ ]:
# ajusta os tipos das colunas numéricas

def str_to_int(c):
    if isinstance(c, int):
        return c
    else:
        try:
            return int(c)
        except:
            return None
  

df.age = df.age.apply(str_to_int)
df.fnlwgt = df.fnlwgt.apply(str_to_int)
df.education_num = df.education_num.apply(str_to_int)
df.capital_gain = df.capital_gain.apply(str_to_int)
df.capital_loss = df.capital_loss.apply(str_to_int)
df.hr_per_week = df.hr_per_week.apply(str_to_int)

df.info()

# Nossas hipóteses:

## *1ª Hipótese: Será que há correlação entre etnia e horas de trabalho?*

In [ ]:
# Verificando quais são as etnias listadas
df.race.unique()

In [ ]:
# Mudando por uma numeração, para poder fazer os cálculos de correlação
dfRaces = df.replace(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other'], 
                     [1, 2, 3, 4, 5])
dfRaces.race.unique()

In [ ]:
# Calculando as correlações, usando Pearson e Spearman
r, p_value = pearsonr(dfRaces.race.values, dfRaces.hr_per_week.values)
display(Markdown('#### Coeficiente de Correlação de Pearson, usando scipy'))
display(Markdown('{:.5f}'.format(r)))

if abs(r) > 0.8:
    display(Markdown('Forte correlação'))
else:
    if abs(r) > 0.5:
        display(Markdown('Correlação média'))
    else:
        display(Markdown('Correlação fraca ou não há correlação'))

from scipy.stats.stats import spearmanr

display(Markdown('#### Coeficiente de Correlação de ranking de Spearman, usando scipy'))
spearman_rho = spearmanr(dfRaces.race.values, dfRaces.hr_per_week.values)[0]
display(Markdown('{:.5f}'.format(spearman_rho)))

Vemos que de fato a correlação para ambos os casos é extremamente baixa

In [ ]:
# Montando um Scatterplot para vasualizar os valores
fig, ax1 = plt.subplots(1, figsize=(15,4))

# Substituindo os valores numéricos pela string de etnia que representam para mostrar no gráfico
Xuniques, Xaxis = np.unique(df['race'], return_inverse=True)
ax1.scatter(Xaxis, dfRaces.hr_per_week, alpha=0.3, marker='.')
ax1.set(xticks=range(len(Xuniques)), xticklabels=Xuniques)

ax1.set_title('Race x Hours per week')

plt.show()

O gráfico nos mostra que existe uma distribuição relativamente equilibrada entre o tempo semanal de trabalho entre cada uma das cinco etnias registradas nos dados, tendo maiores concentrações entre 20 e 60 horas para todas com algumas exceções de distanciando bem dessa média. O gráfico também nos mostra a enorme diferença da quantidade de registros para Brancos em relação às demais.

Como não existe a correlação, a hipótese é rejeitada.

## *2ª Hipótese: Será que empregados de empresas particulares têm maior perda de capital do que servidores públicos em média?*

In [ ]:
# Analisando o que temos no campo de tipo de empregado, nos interessam inicialmente os campos ['State-gov', 'Private', 'Federal-gov','Local-gov']
df.type_employer.unique()

In [ ]:
# Agrupando apenas os dados que nos interessam
dfEmployerType = df.loc[(df['type_employer'] == 'State-gov') | (df['type_employer'] == 'Private') | (df['type_employer'] == 'Federal-gov') | (df['type_employer'] == 'Local-gov')]
dfEmployerType = dfEmployerType[['type_employer','capital_loss']]
dfEmployerType = dfEmployerType.reset_index(drop = True)
dfEmployerType.type_employer.unique()

In [ ]:
# Calculando a média, montando o data frame pelos valores únicos do tipo de empregado
dfEmployerTypeMeans = dfEmployerType.groupby('type_employer').mean()
display(dfEmployerTypeMeans)
dfEmployerTypeMeans.plot(kind='barh', stacked=True, legend=False)

plt.show()

Os valores numéricos e o gráfico nos mostram que, em média, servidores públicos têm maior perda de capital do que empregados de empresas privadas. Sendo assim, a hipótese é rejeitada.

## *3ª Hipótese: Qual é a area de atuação que menos valoriza instrução em termos de ganho de capital?*

In [ ]:
# Começamos mostrando o ganho de capital em relação ao grau de escolaridade
# x salário
# y instrução
# cada conjunto horizontal de pontos, uma área de atuação
fig, (ax1) = plt.subplots(1,1, figsize=(15,4))

Xuniques, Xaxis = np.unique(df['education'], return_inverse=True)
ax1.scatter(df.capital_gain, Xaxis, alpha=0.3, facecolors='none', color='#4466cc')
ax1.set(yticks=range(len(Xuniques)), yticklabels=Xuniques)

plt.show()

O gráfico nos mostra que grau de escolaridade alto não influencia em um maior ganho de capital, assemelhando-se a grau de escolaridade médio. Porém, graus de escolaridades menores possuem ganho de capital pouco expressivo, que é o que se esperaria.

Também podemos ver alguns pontos que se sobressaem positivamente das médias de ganho para graus médios a altos de escolaridade.

In [ ]:
# Montando gráfico de cargos para cargos por nível de escolaridade

plt.subplots(1,1, figsize=(15,4))
df1 = pd.DataFrame(df, columns=('education_num', 'occupation', 'capital_gain'))
df1 = df1.groupby(['occupation','education_num']).mean()
df1 = df1.reset_index()

df1.set_index('education_num',inplace=True)
df1.groupby(['occupation'])['capital_gain'].plot(kind='line',legend=True)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

Nosso objetivo era mostrar no eixo X os nomes dos graus de escolaridade, porém não conseguimos achar uma solução para a forma com que desenvolvemos nosso gráfico até o tempo limite da entrega do exercício. O gráfico mostra como os cargos (cada linha) se distribuem pela quantidade de pessoas (eixo Y) que possuem uma certa instrução (eixo X).

De qualquer forma, vemos que os últimos graus de escolaridade representados no gráfico possuem o maior número de pessoas dentre alguns dos cargos listados. A maioria dos níveis de instrução englobam até 5000 pessoas em todos os cargos.